In [1]:
#make sure to pip install all the required stuff!!!!
from yahoo_fin.stock_info import get_live_price
from yahoo_fin import options as op
import pandas as pd
from datetime import datetime, timedelta
import math
import random
import requests

In [2]:
#get contract and underlying information 
contractInfo = input("Enter the contract symbol for your option: ")
numContracts = int(input('Enter the number of contracts: '))
email = input("Enter your email: ")
#AAPL240719C00105000 <- example contract that you can use as input
tickerSymbol = contractInfo[0:4]

underlyingPrice = get_live_price(tickerSymbol)

year = int(contractInfo[4:6])
month = int(contractInfo[6:8])
day = int(contractInfo[8:10])
expDate = datetime(2000 + year, month, day)

callOrPut = contractInfo[10]
strike = int(contractInfo[11:]) / 1000 

todaysDate = datetime.today()

#get IV for option
def getIV():
  chain = []
  if callOrPut == 'C':
    chain = op.get_calls(tickerSymbol)
  else:
    chain = op.get_puts(tickerSymbol)
  
  chain = pd.DataFrame(chain)
  
  mask = chain['Strike'] == strike
  volPercent = chain[mask].head().to_numpy()[0][10]
  return volPercent[0:len(volPercent) - 1]

ConnectionError: HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/AAPL?period1=7223400&period2=1711050976&interval=1d&events=div%2Csplits (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C588A0A560>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
def getInterestRate(month):
  #https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=2024
  yieldRateMarch11 = {
    1: 5.5,
    2: 5.47,
    3: 5.48,
    4: 5.41,
    6: 5.35,
    12: 4.95,
    36: 4.51,
    60: 4.26,
    84: 4.08, #7 year (what r u doing if ur buying 7 year options)
  }
  dates = [1, 2, 3, 4, 6, 12, 36, 60, 84]
  
  #get closest month to approximate the interest rate
  minDiff = 100000000
  rate = 5.5 #equals month 1 by default
  for date in reversed(dates):
    if abs(date - month) < minDiff:
      minDiff = abs(date - month)
      rate = yieldRateMarch11[date]
  
  return rate #defaut is returning the first month
  
#black scholes formula to estimate delta of option
def getBlackScholesDelta():
  S = float(underlyingPrice) #underlying price
  K = float(strike) #strike
  r = getInterestRate((expDate - todaysDate).days / 30) #risk free return (assume 30 days in a month)
  T = float((expDate - todaysDate).days) #days to maturity
  vol = float(getIV()) / 100 #underlying vol
  #https://www.simtrade.fr/blog_simtrade/option-greeks-delta/
  delta = (math.log(S/K) + T * (r + math.pow(vol, 2) / 2)) / (vol * math.sqrt(T))
  
  if callOrPut == 'P':
    return 1 - delta
  return delta

In [ ]:
history = [] #all purchases (adjustments) to the initial options purchase

def adjust(delta):
  if delta > 0:
    print(f"Sell {delta} shares of underlying")
    buySellUnderlying(-1 * delta)
  elif delta < 0:
    print(f"Buy {delta * -1} shares of underlying")
    buySellUnderlying(-1 * delta)
  else:
    print("No adjustments are required")

def buySellUnderlying(amount):
  history.append([amount, get_live_price(tickerSymbol)])
  data = {
    "recipients": {email}, #enter email address of the user,  
    "subjectIn": "DELTA HEDGE REMINDER",
    "body": "Hello, you should make an adjustment to your position by {amount} shares"
  }
  url = 'http://localhost:3000/email'

  response = requests.post(url, json=data)
  
def getPositionDelta():
  #get delta of shares
  numShares = 0
  for purchase in history:
    numShares += purchase[0]
  
  #add shares delta to options delta
  return getBlackScholesDelta() * numContracts + numShares

In [ ]:
#initial spread
optionDelta = getBlackScholesDelta()
print(f"Your initial spread is: ")
adjust(optionDelta)

Your initial spread is: 
Sell 42.73946538007204 shares of underlying


In [ ]:
initialDaysTillExpiration = (expDate - todaysDate).days
threshold = float(input("Enter the threshold for the delta as decimal between 0.1 and 1(determines how frequently you will adjust): "))

#assuming one adjustment per day
for i in range(initialDaysTillExpiration - 1):   
  todaysDate += timedelta(days=1) #new day!
  underlyingPrice = underlyingPrice + random.randint(-10, 10) #simulate new prices
  
  #check if delta has deviated from 0 by the threshold
  positionDelta = getPositionDelta()
  print("----------------------------------------------------------")
  print(f"Today is {todaysDate.strftime('%m/%d/%Y')} and the delta is {positionDelta}!")
  
  if abs(positionDelta) > threshold:
    adjust(positionDelta)

----------------------------------------------------------
Today is 03/12/2024 and the delta is 170.11524463595322!
Sell 170.11524463595322 shares of underlying
----------------------------------------------------------
Today is 03/13/2024 and the delta is -0.828127311616754!
Buy 0.828127311616754 shares of underlying
----------------------------------------------------------
Today is 03/14/2024 and the delta is -0.8273984592399586!
Buy 0.8273984592399586 shares of underlying
----------------------------------------------------------
Today is 03/15/2024 and the delta is -0.8375200386313963!
Buy 0.8375200386313963 shares of underlying
----------------------------------------------------------
Today is 03/16/2024 and the delta is -0.8449119476426574!
Buy 0.8449119476426574 shares of underlying
----------------------------------------------------------
Today is 03/17/2024 and the delta is -0.8511392010180998!
Buy 0.8511392010180998 shares of underlying
------------------------------------